ResNet50 Pytoch Demo with Profiler

In [ ]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision
import torchvision.transforms as T
import torchvision.datasets as datasets
import torchvision.models as models

In [ ]:
import torch.profiler

In [ ]:
# model in pytorch repo with weights 
model = models.resnet50(pretrained=True)
model.cuda() # load in GPU
cudnn.benchmark = True

In [ ]:
# pre-process images
transform = T.Compose([T.Resize(256), T.CenterCrop(224), T.ToTensor()])
# Dataset load 
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, 
                                        download=True, transform=transform)
# Loading                                 
trainloader = torch.utils.data.DataLoader(trainset, batch_size=16,
                                          shuffle=True)
# calc loss (target and training) - and minimize it
criterion = nn.CrossEntropyLoss().cuda()
# back propagation 
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

device = torch.device("cuda:0")

In [ ]:
# switch to training mode
model.train()

In [ ]:
def output_fn(p):
    p.export_chrome_trace("./trace/resnet50_record/worker0.pt.trace.json")
    
# add context manager around training loop
with torch.profiler.profile(
    activities=[
        torch.profiler.ProfilerActivity.CPU,
        torch.profiler.ProfilerActivity.CUDA],
    schedule=torch.profiler.schedule(
        wait=2, # skip first 2 training steps
        warmup=3, # reach steady and skip few layers, profiling happens ignores results
        active=6), # only profile 6 steps - allows to focus and skip some layers for reducing overhead(even in prod)
    on_trace_ready=output_fn, 
    record_shapes=True
) as p:
    for step, data in enumerate(trainloader, 0):
        print("step:{}".format(step))
        inputs, labels = data[0].to(device=device), data[1].to(device=device)
        with torch.profiler.record_function("# forward #"):
           outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        with torch.profiler.record_function("# backward #"):
          loss.backward()
        optimizer.step()
        
        p.step()
        if step + 1 >= 11:
            break
